# TensorFlow-Keras Optimizers
*by Marvin Bertin*
<img src="../../images/keras-tensorflow-logo.jpg" width="400">

Gradient descent is one of the most popular algorithms to perform optimization and by far the most common way to optimize neural networks.
However there exist many variants of gradient descent each with their strengths and weaknesses.
In this lesson we will cover the most common optimization algorithms provided by TensorFlow and TF-Keras.
You will also gain intuition towards the behaviour of different algorithms for optimizing gradient descent and how to put them to use.

## Basic Gradient Descent
Gradient descent is a way to minimize an objective function $J(\theta)$ parameterized by a model's parameters $\theta \in \mathbb{R}^d$ by updating the parameters in the opposite direction of the gradient of the objective function $\nabla_\theta J(\theta)$ w.r.t. to the parameters. The learning rate $\eta$ determines the size of the steps we take to reach a (local) minimum. In other words, we follow the direction of the slope of the surface created by the objective function downhill until we reach a valley.

<img src="../../images/grad.png" width="600">

Below are examples of optimizers provided by TensorFlow's train module and by TF-Keras:

TensorFlow train module **`tf.train`**

    tf.train.GradientDescentOptimizer
    tf.train.MomentumOptimizer
    tf.train.RMSPropOptimizer
    tf.train.AdadeltaOptimizer
    tf.train.AdagradOptimizer
    tf.train.AdamOptimizer
    tf.train.AdagradDAOptimizer
    
TF-Keras optimizer module **`tf.contrib.keras.optimizers`**

    tf_keras.optimizers.SGD
    tf_keras.optimizers.Adadelta
    tf_keras.optimizers.Adagrad
    tf_keras.optimizers.RMSprop
    tf_keras.optimizers.Adamax
    tf_keras.optimizers.Nadam
    tf_keras.optimizers.Optimizer

## Gradient Descent Variants

There are three variants of gradient descent, which differ in how much data we use to compute the gradient of the objective function. Depending on the amount of data, we make a trade-off between the accuracy of the parameter update and the time it takes to perform an update.

$$\theta = \theta - \eta \cdot \nabla_\theta J( \theta; x; y)$$

- **Batch gradient descent**
    - computes the gradient of the cost function w.r.t. to the parameters θ for the entire training dataset.
- **Stochastic gradient descent**
    - computes a parameter update for each training example x and label y
- **Mini-batch gradient descent**
    - computes an update for every mini-batch of nn training examples
    
Mini-batch gradient descent is typically the algorithm of choice when training a neural network because it combines the best of both worlds
1. updates have lower variances than  stochastic gradient descent, therefore the objective function fluctuates less and the model converges better.
2. the dataset doesn't need to fit in memory like for batch gradient descent, therefore the model can train on very large datasets
3. gradient computation step is faster because the batch is smaller and make use of highly optimized matrix multiplication operation common to state-of-the-art deep learning libraries.

In [1]:
import tensorflow as tf

tf_keras = tf.contrib.keras

In [ ]:
# TensorFlow model inference
model_predictions = MyDeepLearningModel(inputs)

# model loss
cross_entropy_loss = MyLossFunction(labels, model_predictions)

# model evaluation metric
evalution_metric = MyEvaluationMetric(labels, model_predictions)

# model Mini-Batch gradient descent optimizer
# Tensorflow optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

# TF-Keras optimzer
optimizer = tf_keras.optimizers.SGD(lr = learning_rate)

## Short Comings of Mini-Batch Gradient Descent

<img src="../../images/learning_rates.jpg" width="400">

- **Learning rate magnitude**
    - learning rate too large and loss function fluctuates around the minimum preventing convergence
    - learning rate too small and training takes too long to reach convergence
    
- **Learning rate scheduling**
    - annealing - reducing the learning rate according to a pre-defined schedule to allow congergence
    - rigid method that doesn't adapt to a dataset's characteristics
    - requires close monitoring of the training

- **Same learning rate for all parameters**
    - for parse data (some features have low frequency), we might not want to update all parameters equally

- **Challenge of minimizing highly non-convex error functions**
    - escaping suboptimal local minima or saddle points (gradients close to zero in most dimensions) can be difficult

## Momentum Optimizer

SGD tends to oscillate when the loss surface curves much more steeply in one dimension than in another, which are common around local optima.

<img src="../../images/momentum.png" width="600">

Momentum is a method that helps accelerate SGD and almost always enjoys better converge rates on deep networks. In a sense the Momentum optimizer gives potential energy to the update step. This allows the parameter vector to build up velocity in any direction that has consistent gradient. Just like pushing a ball down a hill. The ball accumulates momentum as it rolls downhill, becoming faster and faster on the way.
As a result, we gain faster convergence and reduced oscillation.

Update step:

$$v_t = \gamma v_{t-1} + \eta \nabla_\theta J( \theta)$$
$$\theta = \theta - v_t$$


The momentum term (friction term) $\gamma$ is usually set to 0.9 or a similar value.




In [3]:
# Momentum Optimizers

# Tensorflow optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum)

# TF-Keras optimizer
optimizer = tf_keras.optimizers.SGD(lr=learning_rate, momentum=momentum)

## Nesterov Accelerated Gradient (NAG) Optimizer

Nesterov Momentum is a slightly different version of the momentum update. It enjoys stronger theoretical converge guarantees for convex functions and in practice it also consistenly works slightly better than standard momentum in neural networks.

<img src="../../images/nesterov.jpeg" width="800">

Instead of evaluating gradient at the current position (red circle), we know that our momentum is about to carry us to the tip of the green arrow. With Nesterov momentum we therefore instead evaluate the gradient at this "looked-ahead" position.

$$v_t = \gamma v_{t-1} + \eta \nabla_\theta J( \theta - \gamma v_{t-1} )$$
$$\theta = \theta - v_t$$

This anticipatory update prevents us from going too fast (overshooting), which has significantly increased the performance of RNNs on a number of tasks.

In [ ]:
# Nesterov Optimizers

# Tensorflow optimizer
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=momentum, use_nesterov=True)

# TF-Keras optimizer
optimizer = tf_keras.optimizers.SGD(lr=learning_rate, momentum=momentum, nesterov=True)

## Per-parameter adaptive learning rate methods

So far the optimizers that we've see manipulated the learning rate globally and equally for all parameters. Tuning the learning rates is an expensive process, so there exists other methods that can adaptively tune the learning rates, and even do so per parameter to perform larger or smaller updates depending on their importance. In this section we highlight some common adaptive methods you may encounter in practice.

## Adagrad Optimizer

In Adagrad the learning rate is normalize by the sum of squared gradients on a per-parameter basis. This adaptive optimizer performes larger updates for infrequent and smaller updates for frequent parameters. For this reason, it is well-suited for dealing with sparse data.

Simplified update step:

```
# Assume the gradient dx and parameter vector x
cache += dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)
```

In [ ]:
# Adagrad Optimizers

# Tensorflow optimizer
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)


# TF-keras optimizer
optimizer = tf_keras.optimizers.Adagrad(lr=learning_rate)

## AdaDelta and RMSprop

AdaDelta and RMSprop are both very similar optimizers that improves on Adagrad by instead normalizing the learning rate with the moving average of squared gradients. This reduces the aggressive, monotonically decreasing learning rate found in Adagrad.

Simplified update step:

```
cache = decay_rate * cache + (1 - decay_rate) * dx**2
x += - learning_rate * dx / (np.sqrt(cache) + eps)
```

In [ ]:
# Adadelta and RMSprop Optimizers

# Tensorflow optimizer

# Adadelta
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)

# RMSprop
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)


# TF-keras optimizer

# Adadelta
optimizer = tf_keras.optimizers.Adadelta(lr=learning_rate)

# RMSprop
optimizer = tf_keras.optimizers.RMSprop(lr=learning_rate)

## Adaptive Moment Estimation (Adam) Optimizer

Adam is often the most recommended optimizer for training Deep Neural networks. It combines idea from both RMSProp the Momentum method. It generates a "smooth" estimate (exponentially decaying average) of the gradients' mean and variance. Giving you the best a less noisy gradient and an adaptive learning rates for each parameter.

Simplified update step:

```
m = beta1*m + (1-beta1)*dx
v = beta2*v + (1-beta2)*(dx**2)
x += - learning_rate * m / (np.sqrt(v) + eps)
```

## Visualization of Different Optimizers

<img src="../../images/opt1.gif" width="400" align="left">
<img src="../../images/opt2.gif" width="400" align="left">

Here are other Tensorflow optimizers:

    tf.train.AdagradDAOptimizer
    tf_keras.optimizers.Adamax
    tf_keras.optimizers.Nadam
    
I will leave it to you to explore these optimizers on your own time.

## Next Lesson
### CNN in Tensorflow and TF-Keras
-  You will learn about CNN layers and how they work

<img src="../../images/divider.png" width="100">